In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn import metrics
import pickle

In [2]:
df_full = pd.read_hdf('data/df_full.h5')
df_full.head()

,id,date,available,price,minimum_nights,maximum_nights,host_id,host_response_time,host_is_superhost,host_listings_count,...,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,review,month
0,2595,2022-06-04,0,225.0,30,1125,2845.0,a few days or more,0.0,6.0,...,4.72,4.62,4.76,4.79,4.86,4.41,0.0,3.0,0 11/21/2009\n1 5/28/2009\n2 ...,Jun
1,2595,2022-06-05,0,225.0,30,1125,2845.0,a few days or more,0.0,6.0,...,4.72,4.62,4.76,4.79,4.86,4.41,0.0,3.0,0 11/21/2009\n1 5/28/2009\n2 ...,Jun
2,2595,2022-06-06,0,225.0,30,1125,2845.0,a few days or more,0.0,6.0,...,4.72,4.62,4.76,4.79,4.86,4.41,0.0,3.0,0 11/21/2009\n1 5/28/2009\n2 ...,Jun
3,2595,2022-06-07,0,225.0,30,1125,2845.0,a few days or more,0.0,6.0,...,4.72,4.62,4.76,4.79,4.86,4.41,0.0,3.0,0 11/21/2009\n1 5/28/2009\n2 ...,Jun
4,2595,2022-06-08,0,225.0,30,1125,2845.0,a few days or more,0.0,6.0,...,4.72,4.62,4.76,4.79,4.86,4.41,0.0,3.0,0 11/21/2009\n1 5/28/2009\n2 ...,Jun


In [3]:
df_full.shape

(1036165, 36)

In [4]:
Queens = df_full.copy()
Queens = Queens[Queens['neighbourhood_group_cleansed'] == 'Queens']
Queens.shape

(81030, 36)

In [5]:
Queens.to_hdf(r'data/Queens.h5', key='stage', mode='w') #run twice

C:\Users\matth\anaconda3\lib\site-packages\pandas\core\generic.py:2703: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['host_response_time', 'host_verifications', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'property_type', 'room_type',
       'amenities', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'review', 'month'],
      dtype='object')]

  pytables.to_hdf(


Make dummies


In [6]:
Queens_dummies = pd.get_dummies(Queens, columns=['host_response_time', 'room_type'])

In [7]:
Queens_dummies.corr()['price'].sort_values()

room_type_Private room                  -0.342764
host_identity_verified                  -0.142984
room_type_Shared room                   -0.090931
host_response_time_within a few hours   -0.087238
calculated_host_listings_count          -0.083225
instant_bookable                        -0.038319
host_listings_count                     -0.037789
number_of_reviews                       -0.029600
host_response_time_unknown              -0.017743
host_response_time_a few days or more   -0.011464
host_response_time_within a day         -0.010317
latitude                                -0.009885
minimum_nights                           0.002815
available                                0.032124
availability_365                         0.043416
id                                       0.046101
has_availability                         0.053811
maximum_nights                           0.085111
longitude                                0.093391
host_is_superhost                        0.097559


In [8]:
def tts_borough(df, xvars, dge):
    X = df[xvars]
    y = df['price']
    poly = PolynomialFeatures(include_bias = False, degree = dge)
    X_poly = poly.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_poly, y, random_state=42)
    return X_train, X_test, y_train, y_test

In [9]:
xvars = ['accommodates', 'beds', 'room_type_Entire home/apt', 'room_type_Private room', 'host_identity_verified', 'host_id', 'host_response_time_within an hour', 'host_is_superhost']
X_train, X_test, y_train, y_test = tts_borough(Queens_dummies, xvars, 2)

Let's make a linear regression model


In [10]:
lr = LinearRegression()

In [11]:
cross_val_score(lr, X_train, y_train, cv = 5).mean()

0.6705820176671878

In [12]:
lr.fit(X_train, y_train)

LinearRegression()

In [13]:
prediction = lr.predict(X_test)

The function is

In [14]:
print(f'Model intercept: {lr.intercept_}')
print(f'Model coefficient values: {lr.coef_}')

Model intercept: -59.732709223957045
Model coefficient values: [ 1.35389448e+01  1.99890296e+01 -9.39716620e+00  1.50712110e+01
  5.58696038e+01  1.45256407e-05  1.28657251e+01  4.74354105e+01
  6.86803919e+00 -3.08105071e+00  1.67234415e+01 -9.04045000e-01
 -6.54780636e+01  1.32258276e-06 -1.89553862e+01  4.84081770e+00
 -9.25669070e+00  1.22567146e+01 -1.98256601e+01  4.10688741e+01
  1.04383773e-07  2.72422873e+01 -2.46421861e+01 -1.07610174e+01
 -1.13686838e-13  2.83143254e+01 -6.86241635e-07  3.55737955e-01
  2.09299835e+01  1.54077466e+01  3.23882542e+01 -2.43495600e-06
 -9.60940363e+00  3.96436396e+00  5.60558504e+01 -1.29348137e-05
  3.11770145e+01 -8.40813063e+01 -1.74958917e-13 -5.70300005e-06
  3.81842446e-07  1.27924608e+01 -1.05916274e+01  4.69404739e+01]


The R^2 value is


In [15]:
print(f'Training R2: {lr.score(X_train, y_train)}')
print(f'Testing R2: {lr.score(X_test, y_test)}')

Training R2: 0.6711136176693455
Testing R2: 0.6730646354403718


The mean squared error is


In [16]:
metrics.mean_squared_error(y_test, prediction)

3922.9925580837375

And the root mean squered error is


In [17]:
np.sqrt(metrics.mean_squared_error(y_test, prediction))

62.63379725103483

How about the mean of residuals


In [18]:
residuals = y_test - prediction
np.abs(residuals).mean()

39.74589721731444

This model is ok, but let's see if we can do better. We will start with decision trees.


In [19]:
#Didn't use host ID because it doesn't make sense as a variable
xvars = ['accommodates', 'beds', 'room_type_Entire home/apt', 'room_type_Private room', 'host_identity_verified', 'host_response_time_within an hour', 'host_is_superhost']
X = Queens_dummies[xvars]
y = Queens_dummies['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [20]:
#I tweeked this until I got the best result
dt = DecisionTreeRegressor(random_state=42)

In [21]:
dt.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [22]:
prediction = dt.predict(X_test)

In [23]:
print(f'Score on training set: {dt.score(X_train, y_train)}')
print(f'Score on testing set: {dt.score(X_test, y_test)}')

Score on training set: 0.9229920151689808
Score on testing set: 0.9238511265992795


In [24]:
metrics.mean_squared_error(y_test, prediction)

913.7324867252249

In [25]:
np.sqrt(metrics.mean_squared_error(y_test, prediction))

30.228008315554384

In [26]:
residuals = y_test - prediction
np.abs(residuals).mean()

18.656372874490017

This is a much better model. Let's look at bagging now.

In [50]:
#Didn't use host ID because it doesn't make sense as a variable
xvars = ['accommodates', 'beds', 'room_type_Entire home/apt', 'room_type_Private room', 'host_identity_verified', 'host_response_time_within an hour', 'host_is_superhost']
X = Queens_dummies[xvars]
y = Queens_dummies['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [28]:
#From lesson 6.02
bag = BaggingRegressor(random_state = 42)
bag.fit(X_train, y_train)

BaggingRegressor(random_state=42)

In [29]:
prediction = bag.predict(X_test)

In [30]:
print(f'Score on training set: {bag.score(X_train, y_train)}')
print(f'Score on testing set: {bag.score(X_test, y_test)}')

Score on training set: 0.9229761235119394
Score on testing set: 0.9238294210101952


In [31]:
metrics.mean_squared_error(y_test, prediction)

913.9929384036823

In [32]:
np.sqrt(metrics.mean_squared_error(y_test, prediction))

30.23231612701353

In [33]:
np.sqrt(metrics.mean_squared_error(y_test, prediction))

30.23231612701353

___

Random Forest:

In [34]:
#Didn't use host ID because it doesn't make sense as a variable
xvars = ['accommodates', 'beds', 'room_type_Entire home/apt', 'room_type_Private room', 'host_identity_verified', 'host_response_time_within an hour', 'host_is_superhost']
X = Queens_dummies[xvars]
y = Queens_dummies['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [35]:
rf = RandomForestRegressor(n_estimators = 100)

In [36]:
rf.fit(X_train,y_train)

RandomForestRegressor()

In [37]:
prediction = rf.predict(X_test)

In [38]:
print(f'Score on training set: {rf.score(X_train, y_train)}')
print(f'Score on testing set: {rf.score(X_test, y_test)}')

Score on training set: 0.9229872769617867
Score on testing set: 0.9238636912730694


In [39]:
metrics.mean_squared_error(y_test, prediction)

913.5817195488478

In [40]:
np.sqrt(metrics.mean_squared_error(y_test, prediction))

30.22551438021937

In [41]:
residuals = y_test - prediction
np.abs(residuals).mean()

18.649080140636027

Extra Trees:

In [42]:
#Didn't use host ID because it doesn't make sense as a variable
xvars = ['accommodates', 'beds', 'room_type_Entire home/apt', 'room_type_Private room', 'host_identity_verified', 'host_response_time_within an hour', 'host_is_superhost']
X = Queens_dummies[xvars]
y = Queens_dummies['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [43]:
et = ExtraTreesRegressor(n_estimators = 100)

In [44]:
et.fit(X_train, y_train)

ExtraTreesRegressor()

In [45]:
prediction = et.predict(X_test)

In [46]:
print(f'Score on training set: {et.score(X_train, y_train)}')
print(f'Score on testing set: {et.score(X_test, y_test)}')

Score on training set: 0.9229920151689808
Score on testing set: 0.9238511265992795


In [47]:
metrics.mean_squared_error(y_test, prediction)

913.7324867252249

In [48]:
np.sqrt(metrics.mean_squared_error(y_test, prediction))

30.228008315554384

In [49]:
residuals = y_test - prediction
np.abs(residuals).mean()

18.656372874490017

All the models, except for linear regression, are good models and basically the same. Any one can be used. So I'll go with the decision tree for the streamlit, since that's what I'll use for the other ones.


We will pickle this object to use for our streamlit app

In [51]:
with open('models/queens_pickle.pkl', 'wb') as f:
    pickle.dump(dt, f)